Importamos las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import os

Creamos las variables que guardan las rutas de los archivos fuente de datos para lectura y grabación de los mismos

In [2]:
'''
# Inicializar entorno de rutas en google drive para carga de archivos 
from google.colab import drive
drive.mount('/content/drive')
'''

"\n# Inicializar entorno de rutas en google drive para carga de archivos \nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [3]:
#Nombre del directorio de datos para rutas de ejecucion en ENTORNO LOCAL
PATH = "datoscsv/"
pathname = os.path.join(PATH)

In [4]:
# cargar dataframes Version Local

climadf = pd.read_csv(pathname+"clima_2013-2022.csv",sep=",",decimal=".")
maindf = pd.read_csv(pathname +'Valencia_Estaciones_SinMeteo_1994 a 2022.csv', delimiter=';', decimal=".")
'''
# cargar dataframes Version Colab en Drive
maindf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/Diarios_2013-2019/Diarios_2013-2019.csv",sep=";",decimal=".")
climadf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/clima/CLIMA_2013-2019.csv", delimiter=',', decimal=".")
'''

/tmp/ipykernel_8781/107938001.py:4: DtypeWarning: Columns (11,23,43) have mixed types. Specify dtype option on import or set low_memory=False.
  maindf = pd.read_csv(pathname +'Valencia_Estaciones_SinMeteo_1994 a 2022.csv', delimiter=';', decimal=".")


'\n# cargar dataframes Version Colab en Drive\nmaindf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/Diarios_2013-2019/Diarios_2013-2019.csv",sep=";",decimal=".")\nclimadf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/clima/CLIMA_2013-2019.csv", delimiter=\',\', decimal=".")\n'

In [5]:
climadf.columns

Index(['FECHA ', 'Racha Max (Km/h) ', 'Hora Racha ', 'Veloc (Media)',
       'Precipitación: l/m2', 'T. Máxima ', 'T.Mínima'],
      dtype='object')

In [6]:
maindf.columns

Index(['COD_ESTACION', 'NOM_ESTACION', 'FECHA', 'SO2', 'CO', 'NO', 'NO2',
       'NOx', 'O3', 'PM10', 'PM2.5', 'PM1', 'NH3', 'C6H6', 'C7H8', 'C8H10',
       'Direc.', 'H.Rel.', 'Precip.', 'Pres.', 'R.Sol.', 'Ruido', 'Temp.',
       'UVB', 'Veloc.', 'Veloc.max.', 'As', 'BaA', 'BaP', 'BbFA', 'BjFA',
       'BkFA', 'Cd', 'CH4', 'DahA', 'FA', 'HCT', 'HMN', 'HNM', 'H2S', 'IcdP',
       'Ni', 'Pb', 'PST'],
      dtype='object')

- Aseguramos que el tipo de datos en campo fecha sea `datetime`
- Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [7]:
# Convertir la fecha a formato datetime
climadf['FECHA '] = pd.to_datetime(climadf['FECHA '], format="%d-%m-%Y")
columclima = climadf.columns.to_list()
borra3 = 'FECHA '
borra = 'Hora Racha '
columclima = list(filter((borra3).__ne__,columclima))
columclima = list(filter((borra).__ne__,columclima))
for colcli in columclima:
    climadf[colcli] = climadf[colcli].replace(',','.', regex=True)
    climadf[colcli] = climadf[colcli].apply(pd.to_numeric)

- Eliminamos las columnas cuyo porcentaje de Nan , nulos o similares esta por encima del 90%
- Aseguramos que el tipo de datos en campos de texto y fecha sea el que corresponde

In [8]:
'''
maindf.drop([ 'BaP','BbFA','BjFA','BkFA','PST', 'UVB','H2S', 'HNM', 'IcdP', 'HCT','HMN', 'FA', 'DahA', 'CH4', 'BaA'] , axis=1, inplace= True)
maindf.drop([ 'NH3','C6H6','C7H8','C8H10','As', 'Cd','Ni', 'Pb'] , axis=1, inplace= True)
'''
# Convertir las cadenas a formato string
maindf['COD_ESTACION'] = maindf['COD_ESTACION'].astype(str)
maindf['NOM_ESTACION'] = maindf['NOM_ESTACION'].astype(str)
# Convertir la fecha a formato datetime
#maindf['FECHA'] = pd.to_datetime(maindf['FECHA'], format="%d-%m-%y")
maindf['FECHA'] = pd.to_datetime(maindf['FECHA'], format="%Y-%m-%d")

Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [9]:
columnas = maindf.columns.to_list()
borra1 = "COD_ESTACION"
borra2 = 'NOM_ESTACION'
borra3 = 'FECHA'
columnas = list(filter((borra1).__ne__,columnas))
columnas = list(filter((borra2).__ne__,columnas))
columnas = list(filter((borra3).__ne__,columnas))

In [10]:
for col in columnas:
    maindf[col] = maindf[col].replace(',','.', regex=True)
    maindf[col] = maindf[col].apply(pd.to_numeric)

### Fusion de Dataframes de clima y calidad del aire ###
- El campo comun es la fecha.
- Los datos de las columnas de clima se repetiran en todas las fechas coincidentes del dataframe resultante.
- Al tratarse de lecturas de datos climatológicos de toda la ciudad ( dataframe `climadf`), todas las estationes tendrán los mismos datos climáticos , en caso de que originalmente esas observaciones estén vacios en el dataframe `maindf`. 

In [11]:
df = pd.merge(left=maindf, right=climadf, how='left', left_on='FECHA', right_on='FECHA ')

Eliminamos las filas correspondientes a la estación Valencia Albufera por motivos geograficos ( mediciones fuera del area metropolitana ) y presencia únicamente en algunos años dentro del rango tratado

In [12]:
'''
indexNames = df[ df['COD_ESTACION'] == '46250051' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
'''

"\nindexNames = df[ df['COD_ESTACION'] == '46250051' ].index\n# Delete these row indexes from dataFrame\ndf.drop(indexNames , inplace=True)\n"

En el caso de la temperatura los huecos se rellenarán con la media calculada de las temperaturas máxima y mínima que provienen del dataframe de clima (`climadf`)

`Tmedia = df.loc[:,'TempMax':'TempMin'].mean(axis=1)`

`df.loc[df['Temp'].isna(),'Temp']=Tmedia`

- Eliminamos las columnas repetidas y las que porcentaje de Nan , nulos o similares esta por encima del 80%

In [13]:
# actualizar columna 'Temp.' con 'TEMP' cuando el campo 'Temp.' no tenga valor
Tmedia = df.loc[:,'T. Máxima ':'T.Mínima'].mean(axis=1)
df.loc[df['Temp.'].isna(),'Temp.']=Tmedia
df.loc[df['Precip.'].isna(),'Precip.']=df['Precipitación: l/m2']
df.loc[df['Veloc.'].isna(),'Veloc.']=df['Veloc (Media)']
df.loc[df['Veloc.max.'].isna(),'Veloc.max.']=df['Racha Max (Km/h) ']
df.drop([ 'Precipitación: l/m2','T. Máxima ','Racha Max (Km/h) ', 'Hora Racha ','FECHA ','T.Mínima','Veloc (Media)'] , axis=1, inplace= True)

Esta operación permite una reducción drástica de los porcentajes de valores NaN , nulos o similares.

Aumentando así la fiabiliad en los pasos posteriores.

In [14]:
maindf.isnull().sum()/len(maindf)*100

COD_ESTACION     0.000000
NOM_ESTACION     0.000000
FECHA            0.000000
SO2             21.657619
CO              61.398982
NO              49.363332
NO2             49.459602
NOx             49.335543
O3              48.218025
PM10            75.391776
PM2.5           80.052402
PM1             88.897270
NH3             97.679612
C6H6            95.944779
C7H8            96.229617
C8H10           95.933862
Direc.          83.500233
H.Rel.          88.442720
Precip.         95.601385
Pres.           88.892307
R.Sol.          88.313699
Ruido           77.601008
Temp.           88.159867
UVB             99.760815
Veloc.          83.488324
Veloc.max.      96.652408
As              97.101003
BaA             99.760815
BaP             99.718139
BbFA            99.813416
BjFA            99.815401
BkFA            99.815401
Cd              97.159559
CH4             96.232595
DahA            99.792574
FA              99.819371
HCT             96.255421
HMN             72.739904
HNM         

In [15]:
df.isnull().sum()/len(df)*100

COD_ESTACION     0.000000
NOM_ESTACION     0.000000
FECHA            0.000000
SO2             21.657884
CO              61.399069
NO              49.360406
NO2             49.456667
NOx             49.332619
O3              48.216182
PM10            75.387776
PM2.5           80.049024
PM1             88.897159
NH3             97.678804
C6H6            95.944109
C7H8            96.228924
C8H10           95.933192
Direc.          83.499558
H.Rel.          88.442645
Precip.         75.471136
Pres.           88.892197
R.Sol.          88.313634
Ruido           77.601794
Temp.           68.957099
UVB             99.760834
Veloc.          67.171792
Veloc.max.      75.481060
As              97.099249
BaA             99.760834
BaP             99.718162
BbFA            99.813431
BjFA            99.815416
BkFA            99.815416
Cd              97.157800
CH4             96.232894
DahA            99.792591
FA              99.819385
HCT             96.255719
HMN             72.742068
HNM         

In [16]:
df.columns

Index(['COD_ESTACION', 'NOM_ESTACION', 'FECHA', 'SO2', 'CO', 'NO', 'NO2',
       'NOx', 'O3', 'PM10', 'PM2.5', 'PM1', 'NH3', 'C6H6', 'C7H8', 'C8H10',
       'Direc.', 'H.Rel.', 'Precip.', 'Pres.', 'R.Sol.', 'Ruido', 'Temp.',
       'UVB', 'Veloc.', 'Veloc.max.', 'As', 'BaA', 'BaP', 'BbFA', 'BjFA',
       'BkFA', 'Cd', 'CH4', 'DahA', 'FA', 'HCT', 'HMN', 'HNM', 'H2S', 'IcdP',
       'Ni', 'Pb', 'PST'],
      dtype='object')

In [17]:
df.drop(['PST','HMN','DahA','R.Sol.','Ruido','Pres.','H.Rel.','Direc.','IcdP','UVB','Ni', 'Pb','NH3','C6H6','C7H8','C8H10','H2S', 'HNM','BaP','BbFA', 'As', 'BaA','FA', 'CH4', 'BjFA','BkFA','Cd','HCT',] , axis=1, inplace= True)

In [18]:
df.isnull().sum()/len(df)*100

COD_ESTACION     0.000000
NOM_ESTACION     0.000000
FECHA            0.000000
SO2             21.657884
CO              61.399069
NO              49.360406
NO2             49.456667
NOx             49.332619
O3              48.216182
PM10            75.387776
PM2.5           80.049024
PM1             88.897159
Precip.         75.471136
Temp.           68.957099
Veloc.          67.171792
Veloc.max.      75.481060
dtype: float64

In [19]:
df.columns

Index(['COD_ESTACION', 'NOM_ESTACION', 'FECHA', 'SO2', 'CO', 'NO', 'NO2',
       'NOx', 'O3', 'PM10', 'PM2.5', 'PM1', 'Precip.', 'Temp.', 'Veloc.',
       'Veloc.max.'],
      dtype='object')

In [20]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
SO2,78943.0,12.530661,13.704414,0.0,3.0,7.0,18.0,321.0
CO,38897.0,0.940908,1.206533,0.0,0.2,0.4,1.3,53.6
NO,51028.0,36.020067,47.371074,0.0,6.0,18.0,48.0,623.0
NO2,50931.0,43.603974,30.159836,0.0,21.0,37.0,59.0,599.0
NOx,51056.0,98.563969,98.459603,0.0,30.0,65.0,133.0,1189.0
O3,52181.0,40.093655,20.527114,2.0,24.0,39.0,55.0,193.0
PM10,24801.0,20.758631,12.723948,0.0,12.0,18.0,27.0,209.0
PM2.5,20104.0,12.677965,8.392583,0.0,7.0,11.0,16.0,104.0
PM1,11188.0,9.308286,6.987852,0.0,4.0,7.0,12.0,92.0
Precip.,24717.0,1.264179,9.339254,0.0,0.0,0.0,0.0,936.9


Finalmente guardamos el resultado en un fichero csv para futuras tareas.

In [21]:
# Guarda datos en CSV:
df.to_csv(pathname+'Valencia_TODASEstaciones_CON_Meteo_1994_2022.csv', header=True, sep=';', index=False)